## MS SQL Server Stored Procedures, Functions, and Dates

#### Which day of the week is the busiest?

In [ ]:
SELECT
    -- Select the day of week value for StartDate
	DATENAME(weekday, StartDate) as DayOfWeek,
    -- Calculate TotalTripHours
	SUM(DATEDIFF(second, StartDate, EndDate))/ 3600 as TotalTripHours 
FROM CapitalBikeShare 
-- Group by the day of week
GROUP BY DATENAME(weekday, StartDate)
-- Order TotalTripHours in descending order
ORDER BY TotalTripHours DESC

#### Saturday is pretty busy, Let's look for outliers:

In [ ]:
SELECT
	-- Calculate TotalRideHours using SUM() and DATEDIFF()
  	SUM(DATEDIFF(SECOND, StartDate, EndDate))/ 3600 AS TotalRideHours,
    -- Select the DATE portion of StartDate
  	CONVERT(DATE, StartDate) AS DateOnly,
    -- Select the WEEKDAY
  	DATENAME(WEEKDAY, CONVERT(DATE, StartDate)) AS DayOfWeek 
FROM CapitalBikeShare
-- Only include Saturday
WHERE DATENAME(WEEKDAY, CONVERT(DATE, StartDate)) = 'Saturday' 
GROUP BY CONVERT(DATE, StartDate);

#### How to Declare and set values

In [ ]:
-- Create @ShiftStartTime
DECLARE @ShiftStartTime AS time = '08:00 AM'

-- Create @StartDate
DECLARE @StartDate AS date

-- Set StartDate to the first StartDate from CapitalBikeShare
SET 
	@StartDate = (
    	SELECT TOP 1 StartDate 
    	FROM CapitalBikeShare 
    	ORDER BY StartDate ASC
		)

-- Create ShiftStartDateTime
DECLARE @ShiftStartDateTime AS datetime

-- Cast StartDate and ShiftStartTime to datetime data types
SET @ShiftStartDateTime = CAST(@StartDate AS datetime) + CAST(@ShiftStartTime AS datetime) 

SELECT @ShiftStartDateTime

#### DECLARE A TABLE AND STORE VALUES WITHIN 

In [ ]:
-- Declare @Shifts as a TABLE
Declare @Shifts TABLE(
    -- Create StartDateTime column
	StartDateTime datetime,
    -- Create EndDateTime column
	EndDateTime datetime)
-- Populate @Shifts
INSERT INTO @Shifts (StartDateTime, EndDateTime)
	SELECT '3/1/2018 8:00 AM', '3/1/2018 4:00 PM'
SELECT * 
FROM @Shifts

#### INSERT INTO @TABLE

In [ ]:
-- Declare @RideDates
DECLARE @RideDates TABLE(
    -- Create RideStart
	RideStart date,
    -- Create RideEnd
	RideEnd date)
-- Populate @RideDates
INSERT INTO @RideDates(RideStart, RideEnd)
-- Select the unique date values of StartDate and EndDate
SELECT DISTINCT
    -- Cast StartDate as date
	CAST(StartDate as date),
    -- Cast EndDate as date
	CAST(EndDate as date) 
FROM CapitalBikeShare 
SELECT * 
FROM @RideDates

#### Get the difference in months between now and 1/1/1900. Add zero to that number

In [ ]:
SELECT DATEADD(month, datediff(month, 0, GETDATE()),0 )

#### New data - Human Resources

#### Create a stored procedure in MS SQL Server with two parameters FirstName and LastName to select the name from the database

In [ ]:
USE AdventureWorks2014
GO

CREATE PROCEDURE HumanResources.uspGetEmployeesTest2
@LastName nvarchar(50),
@FirstName nvarchar(50)
AS
    SET NOCOUNT ON; #STOPS THE MESSAGE OF NUMBER OF ROWS AFFECTED
    SELECT FirstName, LastName, Department
    FROM HumanResources.vEmployeeDepartmentHistory
    WHERE FirstName = @FirstName AND LastName = @LastName
        AND EndDate IS NULL;
GO

#### CALL THE STORED PROCEDURE

In [ ]:
EXECUTE HumanResources.uspGetEmployeesTest2 N'Ackerman', N'Pilar';
EXEC HumanResources.uspGetEmployeesTest2 @LastName = N'Ackerman', @FirstName = N'Pilar';

## MySQL Stored Procedures

In [ ]:
DROP PROCEDURE IF EXISTS spCheckFruitStock;
DELIMITER // #specify delimiter to avoid code break
CREATE PROCEDURE SpCheckFruitStock(thisFruit SMALLINT)
BEGIN
    SELECT
        Fruits.Fruit_Id,
        Fruits.Fruit_Name,
        Fruits.Inventory,
        Units.Unit_Name
    FROM 
        Fruits INNER JOIN Units ON
        Fruits.Unit_Id = Units.Unit_Id
    WHERE 
        Fruits.Fruit_Id = thisFruit;
END //

DELIMITER ; #reset delimiter

#### Call MySQL Stored Procedure

In [ ]:
CALL spCheckFruitStock(1);

#### More advanced MySQL Stored Procedure

In [ ]:
DROP PROCEDURE IF EXISTS spCheckFruitStockLevel;

DELIMITER //

CREATE PROCEDURE spCheckFruitStockLevel(
            IN pFruit_Id SMALLINT(5)
            OUT PstockLevel VARCHAR(6) 
                )
BEGIN
    DECLARE stockNumber SMALLINT;
    SELECT 
        Fruits.Inventory into stockNumber
    FROM 
        Fruits INNER JOIN Units ON
        Fruits.Unit_Id = Units.Unit_Id
    WHERE 
        Fruits.Fruit_Id = pFruit_Id
    
    IF stockNumber > 10 THEN
        SET pStockLevel = 'High';
    ELSEIF (stockNumber <=10 AND stockNumber >=5) THEN
        SET pStockLevel = 'Medium';
    ELSEIF (stockNumber <5) THEN
        SET pStockLevel = 'Low - Please Replace Now!';
    END IF;
    
END //
DELIMITER;

In [ ]:
CALL SpCheckFruitStockLevel(1, @stockLevel);
SELECT @stockLevel;